In [32]:
#! /usr/bin/env python
# -*- encoding: UTF-8 -*-
import cv2
import numpy as np 
from naoqi import ALProxy
import qi
import argparse
import sys
import Image
from matplotlib import pyplot as plt

nao_ip = "192.168.0.38"


navigation_service = ALProxy("ALNavigation", nao_ip, 9559)
motion_service = ALProxy("ALMotion", nao_ip, 9559)
tts_service = ALProxy("ALTextToSpeech", nao_ip, 9559)

motion_service.wakeUp() 

tts_service.setLanguage("English")
radius = 1
tts_service.say("Start exploration")
error_code = navigation_service.explore(radius)

if error_code != 0:
    print "Exploration failed."

path = navigation_service.saveExploration()

print "Exploration saved at path: \"" + path + "\""

navigation_service.startLocalization()
navigation_service.navigateToInMap([0., 0., 0.])
o_position = navigation_service.getRobotPositionInMap()
navigation_service.stopLocalization()

result_map = navigation_service.getMetricalMap()
map_width = result_map[1]
map_height = result_map[2]

o_x = round(-o_position[0][1]/result_map[0] + result_map[3][1]/result_map[0])
o_y = round(-o_position[0][0]/result_map[0] - result_map[3][0]/result_map[0])

img = np.array(result_map[4]).reshape(map_width, map_height)
img = (100 - img) * 2.55 # from 0..100 to 255..0
img = np.array(img, np.uint8)
img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

tts_service.say("I got the map")

Exploration saved at path: "/home/nao/.local/share/Explorer/2019-03-22T140236.779Z.explo"


In [33]:
def get_location(event, x,y, flags, param):
    global ix,iy, drawing, mode, c_position

    if event == cv2.EVENT_FLAG_LBUTTON: 
        drawing = True
        cv2.circle(img_display, (x,y), 1, (255,0,0), 1)
#         cv2.imshow('image', img_display)
    elif event == cv2.EVENT_LBUTTONUP:
        x2m = y * result_map[0] - result_map[3][1]
        y2m = x * result_map[0] + result_map[3][0]
        print (x2m,y2m)
        tts_service.say("Start navigate to position" + str(round(x2m,2))+ "," +str(round(y2m,2)))
        
        navigation_service.startLocalization()
        navigation_service.navigateToInMap([-x2m, -y2m, 0.])        
        c_position = navigation_service.getRobotPositionInMap()
        navigation_service.stopLocalization()
        
        tts_service.say("navigation complete")
        print(c_position)
        

       
drawing = False 
mode = True 
ix,iy = -1,-1

img_display = img.copy()

cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.setMouseCallback('image',get_location)

while True:
    drawing = True
    cv2.circle(img_display, (np.uint8(o_x),np.uint8(o_y)), 1, (0,0,255), 1)
    navigation_service.startLocalization()
    c_position = navigation_service.getRobotPositionInMap()
    navigation_service.stopLocalization()
    c_x = round(-c_position[0][1]/result_map[0] + result_map[3][1]/result_map[0])
    c_y = round(-c_position[0][0]/result_map[0] - result_map[3][0]/result_map[0])
    cv2.circle(img_display, (np.uint8(c_x),np.uint8(c_y)), 1, (0,255,0), 1)
    cv2.imshow('image', img_display) 
    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break

cv2.destroyAllWindows()

(-0.5999999642372131, 0.05000004544854164)
[[0.5861988067626953, -0.040131714195013046, -0.06645456701517105], [0.09095054119825363, 0.07926663011312485, 0.07745754718780518]]
(0.05000004544854164, 0.35000004991889)
[[-0.07099311798810959, -0.32690975069999695, -2.490339994430542], [0.07582012563943863, 0.058519598096609116, 0.08841808140277863]]
(0.35000004991889, 4.470348358154297e-08)
[[-0.369905561208725, -0.06524677574634552, 2.691918134689331], [0.0790250301361084, 0.09323091804981232, 0.15167857706546783]]
(1.250000063329935, 0.1000000461935997)
[[-1.209896445274353, -0.09435814619064331, 3.0582730770111084], [0.18865612149238586, 0.2137443721294403, 0.2761625051498413]]
(1.8500000722706318, 0.1000000461935997)
[[-1.8223391771316528, 0.18883174657821655, 2.9596261978149414], [0.39721405506134033, 0.36020758748054504, 0.47350627183914185]]
(1.5000000670552254, -0.6499999649822712)
[[-1.4188786745071411, 0.5510632395744324, 0.8412341475486755], [0.13704945147037506, 0.346224993467